In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
# FRED API
from fredapi import Fred
fred = Fred(api_key='YOUR-KEY')

In [2]:
df = pd.read_pickle('df.pkl')

In [41]:
last_date = df.index.max()-timedelta(14)

In [42]:
# GETTING NEW DATA
mortgage_30 = fred.get_series_first_release('MORTGAGE30US')
treasury_10 = fred.get_series_first_release('WGS10YR')
treasury_2 = fred.get_series_first_release('WGS2YR')
fed = fred.get_series_first_release('FF')
aaa = fred.get_series_first_release('WAAA')
baa = fred.get_series_first_release('WBAA')

In [43]:
# Setting all dates to the Friday of the week
def get_friday(date):
    days_until_friday = 4 - date.dayofweek
    friday = date + pd.Timedelta(days=days_until_friday)
    return friday

mortgage_30.index = mortgage_30.index.map(get_friday)
treasury_10.index = treasury_10.index.map(get_friday)
treasury_2.index = treasury_2.index.map(get_friday)
fed.index = fed.index.map(get_friday)
aaa.index = aaa.index.map(get_friday)
baa.index = baa.index.map(get_friday)

In [44]:
# Getting only new data
new_df = pd.concat([mortgage_30[mortgage_30.index>last_date],treasury_10[treasury_10.index>last_date],
                                                     treasury_2[treasury_2.index>last_date],
                                                      fed[fed.index>last_date],
                                                       aaa[aaa.index>last_date],
                                                        baa[baa.index>last_date]], axis = 1)
new_df.columns = ['M30Y','T10Y','T2Y','FED','AAA','BAA']
new_df = new_df[~new_df['M30Y'].isnull()]
new_df.dropna(inplace = True)
new_df.head()

,M30Y,T10Y,T2Y,FED,AAA,BAA
date,,,,,,
2023-09-29,7.31,4.58,5.06,5.33,5.35,6.35
2023-10-06,7.49,4.75,5.09,5.33,5.55,6.57
2023-10-13,7.57,4.64,5.01,5.33,5.45,6.47
2023-10-20,7.63,4.87,5.14,5.33,5.67,6.69
2023-10-27,7.79,4.87,5.03,5.33,5.71,6.71


In [45]:
def spread(a,b):
    return new_df.copy()[a]-new_df.copy()[b]

# 10Y - 2Y Spread
new_df['T10-2'] = spread('T10Y','T2Y')
# 10Y - FED Spread
new_df['T10-F'] = spread('T10Y','FED')
# AAA - BAA Spread
new_df['A-B'] = spread('AAA','BAA')
# 30Y - FED Spread
new_df['T10-2'] = spread('M30Y','FED')
# AAA - 10Y Spread
new_df['A-T10'] = spread('AAA','T10Y')

In [46]:
new_df

,M30Y,T10Y,T2Y,FED,AAA,BAA,T10-2,T10-F,A-B,A-T10
date,,,,,,,,,,
2023-09-29,7.31,4.58,5.06,5.33,5.35,6.35,1.98,-0.75,-1.0,0.77
2023-10-06,7.49,4.75,5.09,5.33,5.55,6.57,2.16,-0.58,-1.02,0.8
2023-10-13,7.57,4.64,5.01,5.33,5.45,6.47,2.24,-0.69,-1.02,0.81
2023-10-20,7.63,4.87,5.14,5.33,5.67,6.69,2.3,-0.46,-1.02,0.8
2023-10-27,7.79,4.87,5.03,5.33,5.71,6.71,2.46,-0.46,-1.0,0.84
2023-11-03,7.76,4.75,4.97,5.33,5.61,6.62,2.43,-0.58,-1.01,0.86


# Random Forest

In [118]:
import joblib
loaded_rf = joblib.load('rfm.pkl')
pred = loaded_rf.predict(new_df)

rf_rmse = .507579

pred_rf = pd.concat([pd.Series(pred),new_df['M30Y'].iloc[1:].reset_index(drop = True)], axis = 1).dropna()
pred_rf['Upper Bound Estimate'] = pred_rf[0]+rf_rmse
pred_rf['Lower Bound Estimate'] = pred_rf[0]-rf_rmse
pred_rf.columns = ['Prediction','Actual','Upper Bound Estimate','Lower Bound Estimate']

y_test = pred_rf['Actual'].copy()
test_predictions = pred_rf['Prediction'].copy()


from sklearn import metrics
print("MSE:",metrics.mean_squared_error (y_test,test_predictions))
print("RMSE:", np.sqrt(metrics.mean_squared_error (y_test,test_predictions)))
print("RRMSE:",np.sqrt(metrics.mean_squared_error (y_test,test_predictions))/y_test.mean())
print("MAE:", metrics.mean_absolute_error (y_test, test_predictions))
print("RMAE:", metrics.mean_absolute_error (y_test, test_predictions)/y_test.mean())

pred_rf


MSE: 0.7590068082303741
RMSE: 0.8712099679356143
RRMSE: 0.11391343723007509
MAE: 0.8697013911194184
RMAE: 0.11371618607732979


,Prediction,Actual,Upper Bound Estimate,Lower Bound Estimate
0,6.635256,7.49,7.142835,6.127677
1,6.700298,7.57,7.207877,6.192719
2,6.686063,7.63,7.193642,6.178484
3,6.896835,7.79,7.404414,6.389256
4,6.973041,7.76,7.480620,6.465462


In [119]:
# Next Prediction
pred[-1]-rf_rmse, pred[-1], pred[-1]+rf_rmse

(6.238933314869913, 6.746512314869912, 7.254091314869912)

# LSTM

In [203]:
from tensorflow.keras.models import load_model
model0 = load_model('model0/')
loaded_preprocessing_params = joblib.load('preprocessing_params.pkl')
l_rmse = .161951


data_n = (pd.concat([df.iloc[-10:],new_df], axis = 0))

In [207]:
def lstm_prep_new_data(df, window_size):
    df_as_np = df.to_numpy()
    X_new = []
    for i in range(len(df_as_np) - window_size + 1):
        row = [r for r in df_as_np[i:i+window_size]]
        X_new.append(row)
    return np.array(X_new)

window_size = 11
X_new = lstm_prep_new_data(data_n, window_size)

def preprocess_new_data(X_new, training_mean, training_std):
    X_new[:,:,0] = (X_new[:,:,0] - training_mean) / training_std
    return X_new

In [216]:
X_new = preprocess_new_data(X_new, loaded_preprocessing_params['training_mean'], loaded_preprocessing_params['training_std'])

pred_l = model0.predict(X_new.astype('float32')).flatten()

1/1 [==============================] - 0s 16ms/step


In [223]:
predict_lstm = pd.concat([pd.Series(pred_l),new_df['M30Y'].iloc[1:].reset_index(drop = True)], axis = 1).dropna()

predict_lstm['Upper Bound Estimate'] = predict_lstm[0]+l_rmse
predict_lstm['Lower Bound Estimate'] = predict_lstm[0]-l_rmse
predict_lstm.columns = ['Prediction','Actual','Upper Bound Estimate','Lower Bound Estimate']

y_test = predict_lstm['Actual'].copy()
test_predictions = predict_lstm['Prediction'].copy()


print("MSE:",metrics.mean_squared_error (y_test,test_predictions))
print("RMSE:", np.sqrt(metrics.mean_squared_error (y_test,test_predictions)))
print("RRMSE:",np.sqrt(metrics.mean_squared_error (y_test,test_predictions))/y_test.mean())
print("MAE:", metrics.mean_absolute_error (y_test, test_predictions))
print("RMAE:", metrics.mean_absolute_error (y_test, test_predictions)/y_test.mean())

predict_lstm

MSE: 0.08275895614562137
RMSE: 0.28767856393138047
RRMSE: 0.03761487499102778
MAE: 0.28486447143554694
RMAE: 0.03724692356636335


,Prediction,Actual,Upper Bound Estimate,Lower Bound Estimate
0,7.177323,7.49,7.339274,7.015372
1,7.292666,7.57,7.454618,7.130715
2,7.332806,7.63,7.494757,7.170855
3,7.464249,7.79,7.626200,7.302298
4,7.548634,7.76,7.710585,7.386683


In [225]:
# Next Prediction
pred_l[-1]-l_rmse, pred_l[-1], pred_l[-1]+l_rmse

(7.356846874450683, 7.518798, 7.680748874450684)